In [19]:
import chromadb
from chromadb.utils import embedding_functions
from langchain_openai import ChatOpenAI
from langchain_core.messages import (HumanMessage,SystemMessage, FunctionMessage)
from langchain_core.tools import tool
from langchain.tools.render import format_tool_to_openai_function
import json

default_ef = embedding_functions.DefaultEmbeddingFunction()
client = chromadb.PersistentClient(path="contexts")
db = client.get_collection("daniel", embedding_function=default_ef)


@tool
def context_search(question: str):
    """
    This function takes a question as input, queries the database for the context related to the question,
    prints the documents found, and returns these documents.
    
    Args:
        question (str): The question for which the context is to be found.
        
    Returns:
        list: The documents related to the question.
    """
    context = db.query(query_texts=question, n_results=2,)
    print("Using ", context['documents'])
    return context['documents']


tools = [context_search]
functions = [format_tool_to_openai_function(t) for t in tools]

llm = ChatOpenAI(model="gpt-4-1106-preview")
agent = llm.bind_functions(functions)


In [22]:
messages = [
SystemMessage(content=f'''You are an helpful AI Assistant, if user asks questions about Daniel use the context_search function to get context to answer the question'''),
HumanMessage(content="hy")            
]

res = agent.invoke(messages)
if res.additional_kwargs:
    if "function_call" in res.additional_kwargs:
        
        function_name = res.additional_kwargs["function_call"]['name']
        function_args = res.additional_kwargs["function_call"]['arguments']
        
        print("I need to call this function:", function_name)
        print("And pass the following args:", function_args)
    
        function_result = context_search(json.loads(function_args)["question"])
        print("Function result:", function_result)
        
        messages.append(FunctionMessage(content=f"{function_name} : {function_result}",name=function_name ))
        
        final_res = agent.invoke(messages)
        print(final_res.content)
else:
    print(res.content)
        

Hello! How can I assist you today? If you have any questions or need information, feel free to ask.


In [23]:
#PUTTING THINGS TOGETHER

messages = [
    SystemMessage(content=f'''You are an helpful AI Assistant, if user asks questions about Daniel use the context_search function to get context to answer the question'''),           
    ]

def get_response(question):
    messages.append(HumanMessage(content=question)) 
    res = agent.invoke(messages)
    if res.additional_kwargs:
        if "function_call" in res.additional_kwargs:
            
            function_name = res.additional_kwargs["function_call"]['name']
            function_args = res.additional_kwargs["function_call"]['arguments']
            
            print("I need to call this function:", function_name)
            print("And pass the following args:", function_args)
        
            function_result = context_search(json.loads(function_args)["question"])
            print("Function result:", function_result)
            
            messages.append(FunctionMessage(content=f"{function_name} : {function_result}",name=function_name ))
            
            final_res = agent.invoke(messages)
            return final_res.content
    else:
        return res.content




In [24]:
while True:
    question = input("You: ")
    if question == "exit":
        break
    answer = get_response(question)
    print(f"AI: {answer}")

AI: Hello! How can I assist you today? If you have any questions or need information, feel free to ask.
I need to call this function: context_search
And pass the following args: {"question":"what does Dan do?"}
Using  [['Daniel works in tech', 'Daniel is a developer']]
Function result: [['Daniel works in tech', 'Daniel is a developer']]
AI: Daniel is a developer and he works in the tech industry.
